## Loading

Let's load an existing index (that was created with clip index)

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""
from clip_retrieval.clip_back import KnnService, load_clip_indices

In [ ]:
columns = ["url", "caption"]
indices_loaded, indices, device, model, preprocess, mclip_model = load_clip_indices(
    "/home/rom1504/indices_paths.json", True, True, columns, False
)
knn_service = KnnService(
    indices_loaded=indices_loaded,
    device=device,
    model=model,
    preprocess=preprocess,
    columns_to_return=columns,
    metadata_is_ordered_by_ivf=False,
    mclip_model=mclip_model,
)

## Doing a query

Let's extract a subset of the dataset composed mostly of watermarked images

In [ ]:
results = knn_service.query(
    text_input="watermark",
    modality="image",
    indice_name="laion_400m",
    num_images=1000,
    num_result_ids=1000,
)

In [ ]:
import pandas as pd

url_captions = pd.DataFrame([(e["url"], e["caption"]) for e in results], columns=["url", "caption"])

In [ ]:
url_captions

In [ ]:
url_captions.to_parquet("/tmp/mysubset.parquet")

## Downloading

Finally let's download this subset

In [ ]:
!img2dataset --input_format=parquet --url_list=/tmp/mysubset.parquet --output_folder=/tmp/myoutput --processes_count=16 --thread_count=64 --output_format=files --url_col="url" --caption_col="caption"

In [ ]:
!ls /tmp/myoutput/*

In [ ]:
from IPython.display import Image

Image(filename="/tmp/myoutput/00000/0000.jpg")